# Tarea. Notebook BoW
### Vladimir García Loginova

### Se importan paqueterías y se descargan elementos

In [ ]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
nlp = spc.load("es_core_news_sm")


### Lectura de dataset

In [ ]:
df=pd.read_csv("df_mini_HS.csv")

### Creación de listas con oraciones lematizadas y vocabulario de cada texto

In [ ]:
all_voc=[]
ols=[]

for i in range(df.shape[0]):

    #linmpieza de datos
    oracion=df['text'].iloc[i]    
    oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
    oracion = re.sub("http[s]?\://\S+", "", oracion)  # Eliminar enlaces
    oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
    oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
    oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
    oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
    oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
    oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
    oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración 
    oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de exclamación
    oracion = re.sub(r"[-]", "", oracion)  # Eliminar guión
    oracion = re.sub(r"[:""´´''``]", "", oracion)  # Eliminar otros
    
    #tokenización
    tokens = word_tokenize(oracion)

    #stopwords
    spanish_stopwords = stopwords.words('spanish')
    palabras_filtradas = [palabra for palabra in tokens if palabra not in spanish_stopwords]

    #lematización
    lema = nlp(" ".join(palabras_filtradas))
    oracion_lematizada = " ".join([token.lemma_ for token in lema])
    vectorizador = CountVectorizer()
    vectores = vectorizador.fit_transform([oracion_lematizada])
    vocabulario = vectorizador.get_feature_names_out()

    #se guarda en listas las oraciones lematizadas y vocabulario de cada texto
    all_voc.append(vocabulario)
    ols.append(oracion_lematizada)

### Vectores y vocabulario de todos los textos

In [ ]:
# se crea un solo string de todas las oraciones lematizadas
oraciones_lem_pegadas=' '.join(ols)

#se crea el bag of words del vocabulario de todos los textos
vectorizador_all = CountVectorizer()
vectores_all = vectorizador_all.fit_transform([oraciones_lem_pegadas])
vocabulario_all = vectorizador_all.get_feature_names_out()

### Creación de Dataframe con bag of words de todos los textos
Las columnas 1 y 2 del Dataframe final corresponden al texto original y su oración lematizada, respectivamente. 
Las columnas 2 a 191 (longitud del vocabulario de todos los textos) corresponden al vector de cada texto con valores de 0 ó 1, si el texto contiene una palabra del vocabulario global el valor de la celda del vector es 1, caso contrario es 0.
La última columna corresponde al atributo de la clase (0 o 1) del texto original.

In [ ]:
# Se crea matriz de ceros con dimensiones de número de textos x número de palabras del vocabulario global
final_vectors=np.zeros((df.shape[0],vocabulario_all.shape[0]),dtype=int)

# se sustituyen con 1 los indices donde se encuentra la palabra en el arreglo del vocabulario global
match_mask=[]
for ind,v in enumerate(all_voc):
    indexes=np.where(np.isin(vocabulario_all,v))[0]
    final_vectors[ind][indexes]=1

# se crea el dataframe final con los vectores por texto (cada fila corresponde a un texto)
df_bw = pd.DataFrame(final_vectors,columns = vocabulario_all)
#se inserta texto original
df_bw.insert(0,'texto',df['text'].values)
#se inserta oracion lematizada de cada texto
df_bw.insert(1,'oracion_lematizada', ols)
#se inserta atributo de cada clase en ultima columna
df_bw.insert(final_vectors.shape[1]+2,'label',df['label'].values)
#se imprime el dataframe del BoW final
print(df_bw)